In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# course title will be come and encoded after all process

data = pd.read_csv('../input/udemy-courses/udemy_courses.csv').drop(['course_id', 'course_title', 'url'], axis=1)
X = data.drop('is_paid', axis=1)
y = pd.DataFrame(data.is_paid).astype(np.int)

In [ ]:
data.info()

---

In [ ]:
object_based_columns = X.loc[:, ['level', 'published_timestamp', 'subject']]

---

In [ ]:
object_based_columns['level'].drop_duplicates()

In [ ]:
object_based_columns['level'][object_based_columns['level'] == 'All Levels'] = 0
object_based_columns['level'][object_based_columns['level'] == 'Beginner Level'] = 1
object_based_columns['level'][object_based_columns['level'] == 'Intermediate Level'] = 2
object_based_columns['level'][object_based_columns['level'] == 'Expert Level'] = 3
object_based_columns['level'].describe()

---

In [ ]:
object_based_columns['subject'].drop_duplicates()

In [ ]:
onehotencode_arrays = np.zeros([object_based_columns['subject'].shape[0], 4])

onehotencode_arrays[object_based_columns['subject'] == 'Business Finance', 0] = 1
onehotencode_arrays[object_based_columns['subject'] == 'Graphic Design', 1] = 1
onehotencode_arrays[object_based_columns['subject'] == 'Musical Instruments', 2] = 1
onehotencode_arrays[object_based_columns['subject'] == 'Web Development', 3] = 1
print(onehotencode_arrays)

object_based_columns = pd.concat([object_based_columns.drop('subject', axis=1), pd.DataFrame(onehotencode_arrays)], axis=1)

---

In [ ]:
min_year = [2017, -1]
for rowi in range(len(object_based_columns['published_timestamp'])):
    yr = int(object_based_columns.loc[rowi, 'published_timestamp'][:4])
    if min_year[0] >= yr:
        min_year[0] = yr
        min_year[1] = rowi
min_year

In [ ]:
for rowi in range(len(object_based_columns['published_timestamp'])):
    yr = int(object_based_columns.loc[rowi, 'published_timestamp'][:4]) - 2011;
    m = int(object_based_columns.loc[rowi, 'published_timestamp'][5:7])
    d = int(object_based_columns.loc[rowi, 'published_timestamp'][8:10])
    h = int(object_based_columns.loc[rowi, 'published_timestamp'][11:13])
    mi = int(object_based_columns.loc[rowi, 'published_timestamp'][14:16])
    s = int(object_based_columns.loc[rowi, 'published_timestamp'][17:-1])

    yrc = 60 * 60 * 24 * 30 * 12
    mc = 60 * 60 * 24 * 30
    dc = 60 * 60 * 24
    hc = 60 * 60
    mic = 60
    sc = 1

    res = [ yrc * yr
         + mc * m
         + dc * d
         + hc * h
         + mic * mi
         + sc * s ][0]

    object_based_columns.loc[rowi, 'published_timestamp'] = res
object_based_columns['published_timestamp'].describe()    

---

In [ ]:
# end of encoding object type columns
X = pd.concat([X.drop(['level', 'published_timestamp', 'subject'], axis=1), object_based_columns], axis=1)

---

In [ ]:
# num_subcribers, num_reviews, num_lectures, content_duration seem have much extremely upper values

fig, axs = plt.subplots(1, 7, figsize=(30, 4))
columns = X.columns
k = 0
for j in range(7):
    axs[j].scatter(X[columns[k]], range(X.shape[0]), label=columns[k])
    axs[j].legend()
    k += 1
plt.show()

---

In [ ]:
# left after that column: num_reviews, num_lectures, content_duration

plt.figure(figsize=(8, 6))
plt.scatter(X['num_subscribers'], range(data.shape[0]))
plt.show()

In [ ]:
(X['num_subscribers'].value_counts().index > 30000).sum(), X['num_subscribers'].median()

In [ ]:
X.loc[X['num_subscribers'] > 30000, 'num_subscribers'] = X['num_subscribers'].median()

---

In [ ]:
# left after that column: num_lectures, content_duration

plt.figure(figsize=(8, 6))
plt.scatter(X['num_reviews'], range(X.shape[0]))
plt.show()

In [ ]:
(X['num_reviews'].value_counts().index > 3000).sum(), X['num_reviews'].median()

In [ ]:
X.loc[data['num_reviews'] > 3000, 'num_reviews'] = X['num_reviews'].median()

---

In [ ]:
# left after that column: content_duration

plt.figure(figsize=(8, 6))
plt.scatter(X['num_lectures'], range(X.shape[0]))
plt.show()

In [ ]:
(X['num_lectures'].value_counts().index > 280).sum(), X['num_lectures'].median()

In [ ]:
X.loc[X['num_lectures'] > 280, 'num_lectures'] = X['num_lectures'].median()

---

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X['content_duration'], range(X.shape[0]))
plt.show()

In [ ]:
(X['content_duration'].value_counts().index > 28).sum(), X['content_duration'].median()

In [ ]:
X.loc[X['content_duration'] > 28, 'content_duration'] = X['content_duration'].median()

---

In [ ]:
pd.concat([X, y], axis=1).corr()['is_paid'].drop('is_paid', axis=0).sort_values(ascending=False)

In [ ]:
def scaler(X):
    X = (X - X.mean()) / X.max()
    return X
columns = X.columns
for column in columns:
    X.loc[:, column] = scaler(X[column].values)
X

In [ ]:
new_columns = [
    X.price * X.num_lectures, # pos
    X.price * X.content_duration, # pos
    X.num_lectures * X.content_duration, # pos
    
    X.num_subscribers * X.num_reviews, # neg
    
    X.price / X.num_subscribers,
    X.num_subscribers / X.price,
    X.num_lectures / X.num_subscribers,
    X.num_subscribers / X.num_lectures,
    X.content_duration / X.num_subscribers,
    X.num_subscribers / X.content_duration,

    X.price / X.num_reviews,
    X.num_reviews / X.price,
    X.num_lectures / X.num_reviews,
    X.num_reviews / X.num_lectures,
    X.content_duration / X.num_reviews,
    X.num_reviews / X.content_duration
]
for new_column in new_columns:
    X = pd.DataFrame(np.concatenate([X, new_column.values.reshape(-1, 1)], axis=1)).astype(np.float)
X

In [ ]:
X.describe().iloc[:,15:]

In [ ]:
def scaler(X):
    X = (X - X.mean()) / X.max()
    return X
indexes = list(range(15, 27))
for index in indexes:
    X.iloc[:, index] = scaler(X.iloc[:, index].values)
X

In [ ]:
fig, axs = plt.subplots(14, 2, figsize=(8, 45))
k = 0
for i in range(13):
    for j in range(2):
        axs[i, j].scatter(X.iloc[:, i], range(X.shape[0]), label=k)
        axs[i, j].legend()
        k += 1
axs[13, 0].scatter(X.iloc[:, 26], range(X.shape[0]), label=26)
axs[i, j].legend()
plt.show()

---

In [ ]:
from sklearn.decomposition import KernelPCA
pca = KernelPCA(n_components=round(X.shape[1]/5), kernel='rbf')
pca.fit(X)
X = np.concatenate([X, pca.transform(X)], axis=1)
X

---

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=.2, random_state=777)

---

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain.values.reshape(-1,))
ypred = model.predict(Xvalid)

---

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(ypred, yvalid), confusion_matrix(ypred, yvalid)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(estimator=XGBClassifier(n_estimators=1000), X=X, y=y, scoring='accuracy', cv=3, n_jobs=-1)